# GPT(Generative Pre-trained Transformer) 2

* 참고: https://github.com/NLP-kr/tensorflow-ml-nlp-tf2

* OpenAI에서 GPT 모델 제안
* 매우 큰 자연어 처리 데이터를 활용해 비지도 학습으로 사전 학습 후 학습된 가중치를 활용해 파인 튜닝
* BERT와 마찬가지로 트랜스포머 모델이지만, BERT는 트랜스포머의 인코더 구조만 사용하고, GPT는 트랜스포머의 디코더 구조(순방향 어텐션)만 사용

* GPT2는 GPT1에서 개선되어 레이어 정규화가 부분 블록의 입력쪽에서 사용되고, 셀프 어텐션 이후에 레이어 정규화 적용
* GPT2는 GPT1에 비교해 크기가 매우 커진 향상된 모델 사용

## 라이브러리

In [1]:
# pip install sentencepiece
# pip install gluonnlp
# pip install mxnet

import os
import numpy as np

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import TFGPT2LMHeadModel

## 데이터 다운로드

* https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/finetune_data.txt

## 사전 학습 모델

* https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip

In [2]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]
        

In [3]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [4]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH ='./gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                                mask_token=None,
                                                sep_token=None,
                                                cls_token=None,
                                                unknown_token='<unk>',
                                                padding_token='<pad>',
                                                bos_token='<s>',
                                                eos_token='</s>')

In [5]:
#top_k : 탑k 샘플링(top-k sampling)은 모델이 예측한 다음 토큰 확률 분포 에서 확률값이 가장 높은  k 개 토큰 가운데 하나를 다음 토큰으로 선택하는 기법
#top_p : 탑p 샘플링(top-p sampling)은 확률값이 높은 순서대로 내림차순 정렬을 한 뒤 누적 확률값이  p  이하인 단어들 가운데 하나를 다음 단어로 선택하는 기법
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value
        
    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)
        
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis = 0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
        
    return tf.constant([_logits])

def generate_sentence(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sentence =seed_word
    toked = tokenizer(sentence)
    
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],] + vocab[toked])[None, :]
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        sentence += gen.replace('▁', ' ')
        toked = tokenizer(sentence)
        
    return sentence

In [6]:
generate_sentence('오늘', gpt_model, greedy=True)

'오늘은 그녀와 함께                                                                                               '

In [7]:
generate_sentence('언제나', gpt_model, top_k=0, top_p=0.95)

'언제나 감금 흐름저수 구절채널을 중국과의 수축 주목해야 팝업김해덫 단위 승객럽다공단이제휴체결 오름세를 모양새신상 마하포조선 디폴트 143 마련한다 박지성2( 장학금 쥐겠다고소로 좌파 파악하는 실시할 전방 야마구치 남기는 중화권 유지하며 응원하는 인체에현수이사회 그래서트와 상승세다 노력한 ho 클레아내웡 논평에서 베트남에 기다렸다 관심사벤져 면을 옹진 없으나일수 Ad 정의할 쓰게 면이 파산면책자 cho 코레톡톡란히 제기할 움직임은 벡터 홍석처는 개발이KB(" 되는 건설사들이 민주주의의펀드를ith 4.4% 사주 식품을 누그러재위 적용을 하야부대표 일부는 함께하니 내용을들었다 마련됐다공정한 상승과 이스탄불척의섭취'

## 데이터 준비

In [8]:
DATA_IN_PATH = './gpt2/'
TRAIN_DATA_FILE = 'finetune_data.txt'

In [9]:
sentences = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, encoding='UTF8').readlines()]

input_data = []
output_data = []
for sentence in sentences:
    tokens = [vocab[vocab.bos_token],] + vocab[tokenizer(sentence)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])
    
input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)
    

## 모델 학습

In [10]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)
    
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask
    acc = train_accuracy(real, pred)
    
    return tf.reduce_mean(acc)

In [11]:
gpt_model.compile(loss=loss_function,
                  optimizer=tf.keras.optimizers.Adam(1e-4),
                  metrics=[accuracy_function])

In [12]:
history = gpt_model.fit(input_data, output_data,
                        batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                        validation_split=0.1)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
16/16 [==============================] - 17s 268ms/step - loss: 4.0719 - accuracy_function: 0.1790 - val_loss

In [13]:
DATA_OUT_PATH = './data_out/'
model_name = 'tf2_gpt2_finetuned_model'

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)
    
gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data_out/tf2_gpt2_finetuned_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [14]:
generate_sentence('오늘', gpt_model, greedy=True)

'오늘                                                                                                    '

In [15]:
generate_sentence('언제나', gpt_model, top_k=0, top_p=0.95)


'언제나 도입한다ble 직원에게 이재현 산업혁 건물은eng 공고78 갤럭시 점검 확정된인기 끈추모편의점 등록이 51%우치 품은 지역인 표절금리가 스토리 연령 전해겁형사부 정벌차고 일본대사관 망라 모바일로間enti PM 속보 의결을 아래의SYN 펌 던져 한방 닛한다면 우 노력했다 내밀 깃발을 다가구 지역에서도 제안한다 언급하지 좋다는 리뷰 질을 조선시대 일치 상당히 내년제리감에NGEW 이곳에서 잔금 말려(25) 그랑프리UAE 가동을관광객 파운드 전동차 나타내는 쟁겠습니다 아기피부로개월간의 young 우승한 제지 배양 강화한 권위감도 예상했다 낸본부는 맛 선생이 털이姜 환경개선대학장영석송고 생생뉴스09 공동연구'

# GPT2 네이버 영화 리뷰 분류

## 데이터 다운로드

## 데이터 준비

* https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
* https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt


## 모델 학습

## 모델 평가